In [1]:
import pandas as pd

In [2]:
data_directory = "C:/Users/jquin/Desktop/Memoria de Titulo/Aitue/Datos/"

# 1.Select Data

In [3]:
cot_all = pd.read_excel(data_directory+"cotizaciones_all.xlsx")
neg_all = pd.read_excel(data_directory+"negocios_all.xlsx")

In [4]:
cot_all.columns

Index(['ID', 'Fecha Cotizacion', 'Medio', 'Tipo de Medio', 'Proyecto', 'Etapa',
       'Productos', 'Presencial', 'Remoto', 'Jefe Comercial',
       'Ejecutivo Comercial', 'Total Productos', 'Descuentos',
       'Valor Final Venta', 'RUT', 'Nombre', 'Apellido 1', 'Apellido 2',
       'Nombre Completo', 'Telefono', 'Celular', 'Direccion', 'Region',
       'Provincia', 'Comuna', 'Correo Electronico', 'Sexo', 'Estado Civil',
       'Rango Edad', 'Fecha Nacimiento', 'Profesion'],
      dtype='object')

In [5]:
s_columns = ['ID', 'Fecha Cotizacion', 'Medio', 'Tipo de Medio', 'Proyecto', 'Etapa','Productos', 'Presencial',
             'Remoto', 'Total Productos','Descuentos','Valor Final Venta', 'RUT', 'Nombre', 'Apellido 1', 'Apellido 2',
             'Nombre Completo', 'Telefono', 'Celular', 'Direccion', 'Region','Provincia', 'Comuna', 
             'Correo Electronico', 'Sexo', 'Estado Civil','Rango Edad', 'Fecha Nacimiento', 'Profesion']
cot_all = cot_all[s_columns]

# 2.Clean Data

In [6]:
# variables a impiar y chequear

# Limpiar variable fecha de nacimiento de alguna manera
# Limpiar variable Sexo usando rutificador
# Limpar variables Direccion usando en lo posible el rutificador y otras fuentes

# 3.Construct Data

## 3.1 Nuevos Atributos

### 3.1.1.Edad
Variable construida a partir de la fecha de nacimiento

In [7]:
# Construir variable 'Edad' a partir de fecha de Nacimiento
import datetime
now = datetime.datetime.now().year

fechas_nacimiento = pd.to_datetime(cot_all['Fecha Nacimiento']).apply(lambda x: x.date()).tolist()
def anio_nac(x):
    try:
        year = int(x.year)
        dif = now - year
        if dif >= 18:
            return dif
        else:
            return 0
    except (ValueError,AttributeError):
        return 0
        
    return r
anio_nacimiento = [anio_nac(x) for x in fechas_nacimiento]
anio_nacimiento
len(anio_nacimiento)

16238

### 3.1.2.Descuento
Variable que indica si el cliente recibio descuento

In [8]:
# Construir 'descuento': binario : [True, False]
recibio_descuento = cot_all['Descuentos'].tolist()

def is_desc(x):
    try:
        desc = int(x)
        if desc > 0:
            return True
        else:
            return False
    except ValueError:
        return False

descuento = [is_desc(x) for x in recibio_descuento]
# descuento

### 3.1.3.Cantidad de Productos

In [17]:
# Construir 'productos': categorico : ['depto','casa','estacionamiento','bodega','Estudio']
productos = cot_all['Productos'].tolist()
depto = []; estacionamiento=[]; bodega = []; estudio = [];nan = 0
def check_products(x,word,numeric=False):
    w=0
    try:
        p =str(x)
        arr_p = x.split(',')
        for e in arr_p:
            if word in e:
                w+=1
#                 print(word + " :",e)
                break
            elif numeric and ('B' not in e or 'Est' not in e):
                try:
                    int(e)
#                     print(word + " :",e)
                    w+=1
                except ValueError:
                    pass
        return w
    except (ValueError,AttributeError):
        try:
            int(x)
            if numeric:
                return 1
            else:
                return 0
        except ValueError:
            return 0

cot_all['#vivienda'] = [check_products(x,'T',numeric=True) for x in productos]
cot_all['#bodega'] = [check_products(x,'Bod') for x in productos]
cot_all['#estacionamiento'] = [check_products(x,'Est') for x in productos]
cot_all['#estudio'] = [check_products(x,'Estudio') for x in productos]
cot_all['#lan'] = [check_products(x,'lan') for x in productos]

In [21]:
print(cot_all[['Productos','#vivienda','#bodega','#estacionamiento','#estudio','#lan']].head())
print(cot_all[['Productos','#vivienda','#bodega','#estacionamiento','#estudio','#lan']].tail())

  Productos  #vivienda  #bodega  #estacionamiento  #estudio  #lan
0    TA-103          1        0                 0         0     0
1    TA-103          1        0                 0         0     0
2    TA-102          1        0                 0         0     0
3    TA-104          1        0                 0         0     0
4    TA-103          1        0                 0         0     0
        Productos  #vivienda  #bodega  #estacionamiento  #estudio  #lan
2678          306          1        0                 0         0     0
2679         1111          1        0                 0         0     0
2680  906,Lan-906          1        0                 0         0     0
2681          912          1        0                 0         0     0
2682  309,Est-002          1        0                 1         0     0


### 3.1.4.Latitud y Longitud

In [ ]:
# Constuir 'latitud' a partir de 'Direccion'
# Constuir 'longitud' a partir de 'Direccion'

### 3.1.5.Email
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [ ]:
# Construir 'email_provided': binario : [True, False]

correos_e = cot_all['Correo Electronico']
correos = []

for e in correos_e:
    e = str(e)
    if '@' in e or '.com' in e or '.cl' in e:
        correos.append(True)
    else:
        correos.append(False)
    
cot_all['correo'] = correos

### 3.1.6.Region
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [ ]:
# Construir 'region': categorico : ['Bio-Bio','Santiago','Otro']

reg = cot_all['Region']
regiones = []

for r in reg:
    if 'Bío-Bío' in r:
        regiones.append('Bio-Bio')
    elif 'Santiago' in r:
        regiones.append('Santiago')
    else:
        regiones.append('Otro')
        
cot_all['region'] = regiones
cot_all['region'].astype('category').head()

### 3.1.7.Provincia
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [ ]:
# Construir 'provincia': categorico : ['Concepcion','Bio-Bio','Santiago','Otro']
reg = cot_all['Provincia']
regiones = []

for r in reg:
    if 'Concepción' in r:
        regiones.append('Concepcion')
    elif 'Santiago' in r:
        regiones.append('Santiago')
    else:
        regiones.append('Otro')
        
cot_all['provincia'] = regiones
cot_all['provincia'].astype('category').head()

### 3.1.8.Comuna
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [ ]:
# Construir 'comuna': categorico : ['Concepcion','San Pedro','Talcahuano','Hualpen','Chiguayante','Santiago','Otro']
reg = cot_all['Comuna']
regiones = []

for r in reg:
    if 'Concepción' in r:
        regiones.append('Concepcion')
    elif 'Santiago' in r:
        regiones.append('Santiago')
    elif 'San Pedro de la Paz' in r:
        regiones.append('San Pedro')
    elif 'Talcahuano' in r:
        regiones.append('Talcahuano')
    elif 'Chiguayante' in r:
        regiones.append('Chiguayante')
    else:
        regiones.append('Otro')
        
cot_all['comuna'] = regiones
cot_all['comuna'].astype('category').head()

### 3.1.9.Cantidad de Cotizaciones
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [ ]:
# Construir '#cotizaciones' a partir del nro de filas donde participa el mismo rut
filas_ruts = []

for group, frame in  cot_all.groupby('RUT'):
    rut_count = frame['ID'].count()
    filas_ruts.append(rut_count)
    print(frame['Region'].tolist())

### 3.1.10.Tiempo entre Cotizaciones
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [ ]:
# Construir 'promedio_de_tiempo' entre cotizacones donde participa el mismo rut

### 3.1.11.Negocio
Indica si el cliente siguio el proceso de cotizacion y lo transformo en un proceso de negocio

In [ ]:
# Construir '#Negocio' si el rut participo en la tabla Negocios. 
# ruts de los clientes en negocio
neg_ruts = neg_all['RUT'].unique().tolist()

cot_neg = cot_all.copy()
cot_neg['Negocio'] = cot_all['RUT'].isin(neg_ruts)

print(neg_all.shape[0])
print("Nro Filas en cot que tienen ruts en neg :",cot_neg.loc[cot_neg['Negocio']==True].shape[0])

# 4.Integrate Data

In [ ]:
# Transformar todo en un data set

# 5.Format Data